In [2]:
import pandas as pd
import sqlalchemy as sql
import pyodbc
import db_config as config
import urllib

In [9]:
conn = pyodbc.connect(DRIVER='SQL Server',
                 SERVER = config.server_name,
                 UID = config.user,
                 PWD=config.pwd,
                 DATABASE=config.database_name)

In [10]:
query = 'select * from PoC_SP_CustomerTable'
customers = pd.read_sql_query(query,conn)

C:\Users\ramachandran\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [11]:
conn.close()

In [17]:
USCusts = customers[customers['Plant'].str.contains('(US)')==True]

C:\Users\ramachandran\AppData\Local\Temp\ipykernel_20820\191136228.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  USCusts = customers[customers['Plant'].str.contains('(US)')==True]


In [18]:
USCusts

,AccountId,Account,PlantId,Plant,SystemTypeId,SystemType,Name
0,1,AEMETIS ADVANCED FUELS KEYES INC,1,Aemetis Advanced Fuels KeyesCeres CA(US),5,Other,TEST COMPONENT
1,1,AEMETIS ADVANCED FUELS KEYES INC,1,Aemetis Advanced Fuels KeyesCeres CA(US),10053,Closed Loop System,AEM 290
3,1,AEMETIS ADVANCED FUELS KEYES INC,1,Aemetis Advanced Fuels KeyesCeres CA(US),5,Other,test guid match
4,1,AEMETIS ADVANCED FUELS KEYES INC,1,Aemetis Advanced Fuels KeyesCeres CA(US),4,Boiler,Test boiler
5,1,AEMETIS ADVANCED FUELS KEYES INC,1,Aemetis Advanced Fuels KeyesCeres CA(US),5,Other,test1
...,...,...,...,...,...,...,...
439,161,A & S FABRICATING CO,12201,A & S Fabricating Livermore KY (US),1,Cooling Water,Bulk tank 1
447,221,ACCO UNLIMITED,7015,Acco Unlimited Johnston IA (US),1,Cooling Water,Fermentor2
448,221,ACCO UNLIMITED,7015,Acco Unlimited Johnston IA (US),2,Effluent,Fermentor1
449,221,ACCO UNLIMITED,7015,Acco Unlimited Johnston IA (US),3,Influent,Fermentor3


In [21]:
acsize = {}
for i in customers['Account'].unique():
    acsize.update({i:customers['Account'].tolist().count(i)})

In [23]:
df = pd.DataFrame()
df['Acc'] = list(acsize.keys())
df['Size'] = list(acsize.values())

In [31]:
df.sort_values(by='Size', inplace=True, ascending=False)

In [36]:
df = df.head(4)

In [69]:
Top = customers[customers['Account'].isin(df.Acc)]

In [70]:
Top.sort_values(by="AccountId", inplace=True)

C:\Users\ramachandran\AppData\Local\Temp\ipykernel_20820\2682659554.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Top.sort_values(by="AccountId", inplace=True)


In [71]:
Top

,AccountId,Account,PlantId,Plant,SystemTypeId,SystemType,Name
7,2,CC METALS & ALLOYS LLC,2,CC Metals Alloys Calvert City KY (US),9,Paper Machine,Paper machine
37,2,CC METALS & ALLOYS LLC,2,CC Metals Alloys Calvert City KY (US),1,Cooling Water,Large Cooling Tower System
36,2,CC METALS & ALLOYS LLC,2,CC Metals Alloys Calvert City KY (US),9,Paper Machine,test001
35,2,CC METALS & ALLOYS LLC,2,CC Metals Alloys Calvert City KY (US),8,Pulp Washing,Pulp Washing
34,2,CC METALS & ALLOYS LLC,2,CC Metals Alloys Calvert City KY (US),2,Effluent,Headspace provision
...,...,...,...,...,...,...,...
221,40,BLOCKED INTERNATIONAL PAPER,10502,Sylvamo North America Eastover SC (US),3,Influent,Testdev123
220,40,BLOCKED INTERNATIONAL PAPER,1069,IP Auburn ME (US),2,Effluent,Testdev
241,40,BLOCKED INTERNATIONAL PAPER,4586,IP Rome GA (US),3,Influent,5
230,40,BLOCKED INTERNATIONAL PAPER,203,INTERNATIONAL PAPER,1,Cooling Water,Test1


In [73]:
Recs = Top.to_dict('records')

In [121]:
%%time
a = []
for r in Recs:
    pump_c = random.randint(1,3)
    for i in range(1, pump_c+1):
        k=r
        k.update({'Pump':str(k['Name'])+'_'+str(i)})
        k.update({random.choice(chemicals):random.randint(50,200)})
        a.append(k)

CPU times: total: 0 ns
Wall time: 707 µs


In [85]:
df = pd.DataFrame.from_dict(a)

In [89]:
df.rename(columns={"Name": "Product"},inplace=True)

In [95]:
df = df[(df['Product'].str.contains('test')==False) & (df['Product'].str.contains('Test')==False)]

In [106]:
df = df.reset_index()

In [108]:
df.drop(columns='index', inplace=True)

In [109]:
df

,AccountId,Account,PlantId,Plant,SystemTypeId,SystemType,Product,Pump
0,2,CC METALS & ALLOYS LLC,2,CC Metals Alloys Calvert City KY (US),9,Paper Machine,Paper machine,Paper machine_1
1,2,CC METALS & ALLOYS LLC,2,CC Metals Alloys Calvert City KY (US),1,Cooling Water,Large Cooling Tower System,Large Cooling Tower System_3
2,2,CC METALS & ALLOYS LLC,2,CC Metals Alloys Calvert City KY (US),1,Cooling Water,Large Cooling Tower System,Large Cooling Tower System_3
3,2,CC METALS & ALLOYS LLC,2,CC Metals Alloys Calvert City KY (US),1,Cooling Water,Large Cooling Tower System,Large Cooling Tower System_3
4,2,CC METALS & ALLOYS LLC,2,CC Metals Alloys Calvert City KY (US),8,Pulp Washing,Pulp Washing,Pulp Washing_3
...,...,...,...,...,...,...,...,...
192,40,BLOCKED INTERNATIONAL PAPER,12216,KBG IP Pensacola Cantonment FL (US),5,Other,Headspace Skid QA,Headspace Skid QA_2
193,40,BLOCKED INTERNATIONAL PAPER,12216,KBG IP Pensacola Cantonment FL (US),5,Other,Headspace Skid QA,Headspace Skid QA_2
194,40,BLOCKED INTERNATIONAL PAPER,4586,IP Rome GA (US),3,Influent,5,5_3
195,40,BLOCKED INTERNATIONAL PAPER,4586,IP Rome GA (US),3,Influent,5,5_3


In [118]:
chemicals = ['ChemA', 'ChemB', 'ChemC', 'ChemD', 'ChemE']

In [116]:
import random

In [117]:
random.choice(chemicals)    

'ChemD'

In [120]:
random.random()

0.668745715709343

In [122]:
new = pd.DataFrame.from_dict(a)

In [129]:
import numpy as np
new.replace(np.NaN, 0,inplace=True)

In [131]:
new.columns

Index(['AccountId', 'Account', 'PlantId', 'Plant', 'SystemTypeId',
       'SystemType', 'Name', 'Pump', 'ChemC', 'ChemB', 'ChemD', 'ChemA',
       'ChemE'],
      dtype='object')

In [136]:
sums = new[['Account', 'Plant', 'ChemA', 'ChemB', 'ChemC', 'ChemD', 'ChemE']].groupby(['Account', 'Plant']).sum()

In [137]:
sums.reset_index()

,Account,Plant,ChemA,ChemB,ChemC,ChemD,ChemE
0,BLOCKED INTERNATIONAL PAPER,INTERNATIONAL PAPER,0.0,0.0,0.0,101.0,316.0
1,BLOCKED INTERNATIONAL PAPER,IP Auburn ME (US),588.0,0.0,312.0,0.0,0.0
2,BLOCKED INTERNATIONAL PAPER,IP Cedar Rapids IA (US),657.0,1040.0,1215.0,1610.0,447.0
3,BLOCKED INTERNATIONAL PAPER,IP Rome GA (US),591.0,408.0,0.0,0.0,0.0
4,BLOCKED INTERNATIONAL PAPER,IP Valliant OK (US),0.0,0.0,0.0,68.0,0.0
5,BLOCKED INTERNATIONAL PAPER,International Paper Amarillo TX (US),501.0,102.0,426.0,0.0,441.0
6,BLOCKED INTERNATIONAL PAPER,International Paper Butler IN (US),186.0,126.0,0.0,0.0,0.0
7,BLOCKED INTERNATIONAL PAPER,International Paper Moses Lake WA (US),394.0,424.0,178.0,0.0,0.0
8,BLOCKED INTERNATIONAL PAPER,KBG IP Pensacola Cantonment FL (US),0.0,0.0,0.0,290.0,284.0
9,BLOCKED INTERNATIONAL PAPER,Sylvamo North America Eastover SC (US),172.0,0.0,0.0,626.0,134.0


In [134]:
new

,AccountId,Account,PlantId,Plant,SystemTypeId,SystemType,Name,Pump,ChemC,ChemB,ChemD,ChemA,ChemE
0,2,CC METALS & ALLOYS LLC,2,CC Metals Alloys Calvert City KY (US),9,Paper Machine,Paper machine,Paper machine_1,151.0,0.0,0.0,0.0,0.0
1,2,CC METALS & ALLOYS LLC,2,CC Metals Alloys Calvert City KY (US),1,Cooling Water,Large Cooling Tower System,Large Cooling Tower System_1,0.0,71.0,0.0,0.0,0.0
2,2,CC METALS & ALLOYS LLC,2,CC Metals Alloys Calvert City KY (US),9,Paper Machine,test001,test001_3,0.0,130.0,59.0,0.0,0.0
3,2,CC METALS & ALLOYS LLC,2,CC Metals Alloys Calvert City KY (US),9,Paper Machine,test001,test001_3,0.0,130.0,59.0,0.0,0.0
4,2,CC METALS & ALLOYS LLC,2,CC Metals Alloys Calvert City KY (US),9,Paper Machine,test001,test001_3,0.0,130.0,59.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,40,BLOCKED INTERNATIONAL PAPER,4586,IP Rome GA (US),3,Influent,5,5_3,0.0,136.0,0.0,197.0,0.0
219,40,BLOCKED INTERNATIONAL PAPER,4586,IP Rome GA (US),3,Influent,5,5_3,0.0,136.0,0.0,197.0,0.0
220,40,BLOCKED INTERNATIONAL PAPER,203,INTERNATIONAL PAPER,1,Cooling Water,Test1,Test1_1,0.0,0.0,101.0,0.0,0.0
221,40,BLOCKED INTERNATIONAL PAPER,203,INTERNATIONAL PAPER,3,Influent,Test5,Test5_2,0.0,0.0,0.0,0.0,158.0
